## **Agata Semerjak, 402426**  
gr 1a, środa 14:45  
**lab 4** 24.03.2021

**Zadanie 1.**

In [6]:
def print_matrix(M):
    print("%3s" % ' ', end='')
    print("%3s" % ' | ', end='')
    for i in range(len(M)):
        print("%7s" % str(i), end='')
    print("\n")
    print("-"*(7*len(M) + 6))
    for r in range(len(M)):
        print("%3s" % str(r), end='')
        print("%3s" % ' | ', end='')
        for c in range(len(M)):
            print("%7s" % str(M[r][c]), end='')
        print("\n")

In [7]:
import numpy as np

def Dijkstra(G, s = 0):
    #G to macierz wag
    p = {} #funkcja przyporządkowująca wierzchołkowi jego poprzednika w ścieżce
    d = {} #funkcja przyporządkowująca wierzchołkowi aktualny koszt osiągniecia go z wierzchołka s
    N = {} #funkcja przyporządkowująca wierzchołkowi jego listę sąsiedztwa
    route = {}

    for i in range(len(G)):
        N[i] = []
        route[i] = []
        for j in range(len(G)):
            if i != j and G[i][j]:
                N[i].append(j) #tworzenie list sąsiedztwa
                
    for u in range(len(G)): #początkowe ustawienie funkcji p i d dla wszystkich wierzchołków
        p[u] = 0
        d[u] = np.inf
 
    Q = [i for i in range(len(G))] #stworzenie zbioru wierzchołków 
    d[s] = 0 #"obsłużenie" pierwszego wierzchołka czyli rozpoczęcie tworzenia ścieżki od niego
    Q.remove(s) #usunięcie pierwszego wierzchołka ze zbioru
    u_ = s #u_ to ostatnio wybrany wierzchołek

    while Q != []: 
        for u in Q:
            if u in N[u_]:
                if d[u_] + G[u_][u] < d[u]:
                    d[u] = d[u_] + G[u_][u] #ustawienie prawidłowych wartości d i p dla każdego wierzchołka, który jest sąsiedem u_ (czyli poprzednio wybranego)
                    p[u] = u_
                    if s != u_:
                        route[u].append(u_)
        
        min_val = np.inf
        for key, value in d.items(): #znalezienie u, dla którego d[u] jest najmniejsze
            if value < min_val and key in Q:
                min_val = value
                umin = key
        u_ = umin
        Q.remove(u_)

    return d, route


def Bellman_Ford(G, s = 0):
    #G to macierz wag
    p = {} #funkcja przyporządkowująca wierzchołkowi jego poprzednika w ścieżce
    d = {} #funkcja przyporządkowująca wierzchołkowi aktualny koszt osiągniecia go z wierzchołka s
    #route = {} #funkcja układająca drogę, jednak została zakomentowana, ze względu na to, że algorytm jest używany jedynie w algorytmie Johnsona
    E = [] #zbiór krawędzi grafu
    for i in range(len(G)):
        #route[i] = []
        for j in range(len(G)):
            if G[i][j] is not None:
                E.append((i, j, G[i][j])) #uzupełnianie krawędzi

    for u in range(len(G)): #początkowe ustawienie funkcji d i p dla wszystkich wierzchołków
        p[u] = 0
        d[u] = np.inf
 
    d[s] = 0 #"obsłużenie" pierwszego wierzchołka czyli rozpoczęcie tworzenia ścieżki od niego

    for i in range(len(G) - 1):
        for u, v, w in E:
            if d[u] + w < d[v]:
                d[v] = d[u] + w

    for u, v, w in E:
        if d[u] + w < d[v]: #wykrycie neatywnego cyklu
            raise ValueError("Negative Cycle!")
    
    '''for v in range(len(G)): #stworzenie drogi
        temp_v = v
        while p[temp_v] != s:
            temp_v = p[temp_v]
            route[v].append(temp_v)
        route[v].reverse()'''

    return d #, route


def Johnson(G):
    G.append([0 for i in range(len(G)+1)])
    for i in range(len(G)):
        G[i].append(None)
    G[len(G)-1][len(G)-1] = None #dodaję dodatkowy wierzchołek i tworzę odpowiednie połączenia - tzn z wierzchołka można dotrzeć wszędzie, ale z nikąd nie można dotrzeć do tego wierzchołka

    modified_weights = Bellman_Ford(G, s=len(G)-1) #uruchomienie Bellmana Forda dla dodanego wierzchołka

    G.pop()
    for i in range(len(G)):
        G[i].pop() #usunięcie dodanego wierzchołka i wszystkich połączeń

    for i in range(len(G)):
        for j in range(len(G)):
            if G[i][j] is not None:
                G[i][j] += (modified_weights[i] - modified_weights[j]) #zmodyfikowanie wag wg wzoru
                
    distances = [None for u in range(len(G))] #macierz kosztów
    routes = [None for u in range(len(G))] #macierz dróg
    


    for u in range(len(G)):
        distances[u], routes[u] = Dijkstra(G, s = u) #uruchomienie Djikstry dla każdego wierzchołka

    for u in range(len(G)):
        for v in range(len(G)):
            delta_h = modified_weights[u] - modified_weights[v]
            distances[u][v] = distances[u][v] - (modified_weights[u] - modified_weights[v]) #odtworzenie oryginalnych długości
            if G[u][v] is not None:
                G[u][v] -= delta_h


    return (distances, routes)

def print_johnson(result):
    distances, routes = result
    print("\n")
    for i in range(len(distances)):
        print("Wierzchołek startowy:", i)
        for j in range(len(distances)):
            print("odległość od wierzchołka nr", j, ':', distances[i][j], '. Wierzchołki przez które należy przejść:', routes[i][j])
        print("\n")


**Zadanie 2.**

Algorytm powinien działać dla grafów skierowanych zarówno z ujemnymi, jak i dodatnimi wagami. Algorytm przerwie działanie gdy otrzyma graf, w którym występuje cykl ujemny.

In [8]:
G1 = [[None, 5, None, 9, None, None, 3, None, 3, 3],
    [5, None, 9, None, 8, 6, None, 7, None, None],
    [None, 9, None, 9, 4, None, 5, 3, None, 2],
    [9, None, 9, None, None, None, 8, None, None, 9],
    [None, 8, 4, None, None, 2, 1, None, 7, 5],
    [None, 6, None, None, 2, None, 6, None, None, None],
    [3, None, 5, 8, 1, 6, None, 9, 5, 6],
    [None, 7, 3, None, None, None, 9, None, 8, None],
    [8, 7, 3, None, 6, None, None, None, 8, None],
    [None, None, 3, 9, None, 8, 9, None, 7, None]]
print("G1 = ")
print_matrix(G1)
d3 = Johnson(G1)
print_johnson(d3)

G1 = 
    |       0      1      2      3      4      5      6      7      8      9

----------------------------------------------------------------------------
  0 |    None      5   None      9   None   None      3   None      3      3

  1 |       5   None      9   None      8      6   None      7   None   None

  2 |    None      9   None      9      4   None      5      3   None      2

  3 |       9   None      9   None   None   None      8   None   None      9

  4 |    None      8      4   None   None      2      1   None      7      5

  5 |    None      6   None   None      2   None      6   None   None   None

  6 |       3   None      5      8      1      6   None      9      5      6

  7 |    None      7      3   None   None   None      9   None      8   None

  8 |       8      7      3   None      6   None   None   None      8   None

  9 |    None   None      3      9   None      8      9   None      7   None



Wierzchołek startowy: 0
odległość od wierzchołka nr 0 : 0

Algorytm działa poprawnie dla grafu z dodatnimi wagami

In [9]:
G2 = [[None, -5, None, -9, None, None, -3, None, 3, 3],
    [-5, None, 9, None, 8, 6, None, 7, None, None],
    [None, 9, None, 9, 4, None, 5, 3, None, 2],
    [-9, None, 9, None, None, None, -8, None, None, 9],
    [None, 8, 4, None, None, 2, 1, None, 7, 5],
    [None, 6, None, None, 2, None, 6, None, None, None],
    [3, None, 5, 8, 1, 6, None, 9, 5, 6],
    [None, 7, 3, None, None, None, 9, None, 8, None],
    [8, 7, 3, None, 6, None, None, None, 8, None],
    [None, None, 3, 9, None, 8, 9, None, -7, None]]

d4 = Johnson(G2)

print_johnson(d4)

ValueError: Negative Cycle!

Zgodnie z oczekiwaniami został wykryty ujemny cykl.

In [10]:
G3 = [[None, -5, None, 9, None, None, -3, None, 3, 3],
    [5, None, 9, None, 8, 6, None, 7, None, None],
    [None, 9, None, 9, 4, None, 5, 3, None, 2],
    [9, None, 9, None, None, None, -8, None, None, 9],
    [None, 8, 4, None, None, 2, 1, None, 7, 5],
    [None, 6, None, None, 2, None, 6, None, None, None],
    [3, None, 5, 8, 1, 6, None, 9, 5, 6],
    [None, 7, 3, None, None, None, 9, None, 8, None],
    [8, 7, 3, None, 6, None, None, None, None, None],
    [None, None, 3, 9, None, 8, 9, None, 7, None]]
d4 = Johnson(G3)

print_johnson(d4)



Wierzchołek startowy: 0
odległość od wierzchołka nr 0 : 0 . Wierzchołki przez które należy przejść: []
odległość od wierzchołka nr 1 : 10 . Wierzchołki przez które należy przejść: [8]
odległość od wierzchołka nr 2 : 6 . Wierzchołki przez które należy przejść: [8]
odległość od wierzchołka nr 3 : 9 . Wierzchołki przez które należy przejść: []
odległość od wierzchołka nr 4 : 9 . Wierzchołki przez które należy przejść: [8]
odległość od wierzchołka nr 5 : 16 . Wierzchołki przez które należy przejść: [9]
odległość od wierzchołka nr 6 : 10 . Wierzchołki przez które należy przejść: [2, 4]
odległość od wierzchołka nr 7 : 9 . Wierzchołki przez które należy przejść: [2]
odległość od wierzchołka nr 8 : 3 . Wierzchołki przez które należy przejść: []
odległość od wierzchołka nr 9 : 8 . Wierzchołki przez które należy przejść: [2]


Wierzchołek startowy: 1
odległość od wierzchołka nr 0 : 23 . Wierzchołki przez które należy przejść: [3, 8]
odległość od wierzchołka nr 1 : 0 . Wierzchołki przez które n

Niestety po wprowadzeniu ujemnych wag, algorytm przestaje działać poprawnie. Pomimo kilkunastogodzinnego szukania błędu, nie jestem w stanie stwierdzić, gdzie leży przyczyna. Wszystkie algorytmy wydają się byc zaimplementowane poprawnie. Bellman-Ford i Dijkstra samodzielnie również działają poprawnie.

**Zadanie 3.**
  
Złożoność obliczeniowa algorytmu:
O(n^2log(n) + en), gdzie n jest liczbą wierzchołków, a e liczbą krawędzi. Najbardziej kosztowne czasowo jest wywołanie algorytmy Dijkstry dla każdego wierzchołka. Dla grafu rzadkiego (optymistyczny przypadek) algorytm Johnsona jest szybszy niż Floyda-Warshalla.  
Złożoność pamięciowa jest zależna od wielkości grafu, jednak przez fakt, że jest on reprezentowany jako macierz, to puste połączenia również zajmują miejsce. Nie ma więc znaczenia czy posiada wiele wierzchołków/krawędzi.
